In [3]:
import pandas as pd 
import numpy as np
import time 
import sys; 
print(sys.path)
from pandarallel import pandarallel

from matplotlib import pyplot as plt

# Add directory above current directory to path

if not ".." in sys.path:
    sys.path.insert(0, '..')
    
%load_ext autoreload
%autoreload 2

['/Users/nicholasjunge/Studies/Master Mathematics Munich/Masterarbeit/codebase/notebooks', '/Users/nicholasjunge/anaconda3/lib/python37.zip', '/Users/nicholasjunge/anaconda3/lib/python3.7', '/Users/nicholasjunge/anaconda3/lib/python3.7/lib-dynload', '', '/Users/nicholasjunge/.local/lib/python3.7/site-packages', '/Users/nicholasjunge/anaconda3/lib/python3.7/site-packages', '/Users/nicholasjunge/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/nicholasjunge/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/nicholasjunge/.ipython']


ModuleNotFoundError: No module named 'pandarallel'

## Step 1: Sanitizing the user base

In [2]:
#passing a list of datatypes for the columns of customer base
customer_dtypes = {'ISACTIVE': str, "CUST_CODE": str, 'CLASS_CODE': str, 'BUSINESS_CODE': str, 'SEX': str, 'NO_MAIL': str, 
                   'NO_SMS': str, 'COMPANY_CUST_CODE': str, 'IS_DROP_IN_CUSTOMER': str}

#list of columns that will be parsed as dates
customer_date_cols = ['EXPIRE_DATE', 'LAST_VISIT', 'DATE_SAVED', 'DATE_LAST_MODIFIED', 'RENEW_DATE', 'PAUSE_START', 'PAUSE_END', 'MEMBER_SINCE']

#first row in RES_CARD.xlsx is a nonsense entry, so skip it. HEADER COUNTS --> FIRST ROW HAS INDEX 1
rows_to_skip = [1]

customer_base = pd.read_excel('../DATA_GYM/RES_CARD.xlsx', sheet_name='Sheet1', skiprows=rows_to_skip, dtype=customer_dtypes, parse_dates=customer_date_cols, index_col=2)

The reservation card imported above is a description of the customer base of all the gyms. Some of the most important columns are:

CUST_CODE (string, format: int1DOTint2): Unique customer identifier. Perhaps translate to int for an easier handling? !! Some entries are screwed up, having "~del_cust" (deleted customer?) as CUST_CODE -> remove those from analysis

EXPIRE_DATE (datetime object, format YYYY-MM-DD): Date of contract expiry.

MEMBER_SINCE (datetime object, format YYYY-MM-DD): Date of contract signing. Sometimes day, sometimes second accuracy -> maybe truncate entries to day format.

--> The membership duration for a given customer cust_id is then just reservation_card[cust_id][EXPIRE_DATE] - reservation_card[cust_id][MEMBER_SINCE].

IS_ACTIVE (bool, encoded as F or T): Whether the contract is still active. For these, the expiry date is set to December 31st, 2100.

SEX (string): Gender of customer. Either "M" (male) or "F" (female).

In [ ]:
customer_bool_cols = ['ISACTIVE', 'NO_SMS', 'NO_MAIL', 'IS_DROP_IN_CUSTOMER']

#dict object mapping the boolean columns 
bool_converter = {'T': True, 'F': False}

#convert boolean columns to bools 
for col in customer_bool_cols:
    customer_base[col] = customer_base[col].map(bool_converter) 

The columns "EXPIRE_DATE" and "PAUSE_START" contain strings that let the datetime parsing fail. Therefore, we run it a second time with coercion (that is, putting NaT (not a timestamp) whenever an unparseable string occurs).

In [ ]:
date_cols_with_errs = ["EXPIRE_DATE", "PAUSE_START"]

for col in date_cols_with_errs:
    #makes the parser convert wrong timestamps into NaT (not a timestamp)
    customer_base[col] = pd.to_datetime(customer_base[col], errors='coerce')

In [ ]:
customer_base

In [ ]:
#check how many customers do not have contract start / end dates -> for these, we cannot do frequency analysis
no_expiry = customer_base[customer_base["EXPIRE_DATE"].isna()]
no_start = customer_base[customer_base["MEMBER_SINCE"].isna()]

print(len(no_expiry.index))
print(len(no_start.index))

We end up having 22 entries without contract expiry information. Since we cannot do anything with them without diving deeper into the excel sheet, we drop them from our dataframe.

In [ ]:
#no inplace=True argument because we create another dataframe
sanitized_customer_base = customer_base.dropna(subset=["EXPIRE_DATE"])
print(sanitized_customer_base[sanitized_customer_base["EXPIRE_DATE"].isna()])

In [ ]:
sanitized_customer_base

Now we can get the contract duration in days by simply using the formula CONTRACT_DURATION = EXPIRE_DATE $-$ MEMBER_SINCE, using the column names in our DataFrame. We add this as a new column called "CONTRACT_DURATION".
Unfortunately, some users have the placeholder date "1899-12-30" for the MEMBER_SINCE entry, which means we will not get a sensible contract duration period from them.

In [ ]:
#drop users with nonsense "MEMBER_SINCE" entries
sanitized_customer_base = sanitized_customer_base[sanitized_customer_base["MEMBER_SINCE"] != "1899-12-30"]

#sanitized_customer_base.MEMBER_SINCE.describe()

inactive_users = sanitized_customer_base[sanitized_customer_base.ISACTIVE == False]

weird_inactive_users = inactive_users[sanitized_customer_base.EXPIRE_DATE > "2020-01-01"]

sanitized_customer_base.drop(weird_inactive_users.index, inplace=True)
#drop users with nonsense "EXPIRE_DATE" entries

sanitized_customer_base = sanitized_customer_base[sanitized_customer_base["EXPIRE_DATE"] > "2000-01-01"] 

sanitized_customer_base.MEMBER_SINCE.describe()
#sanitized_customer_base = sanitized_customer_base[inactive_users["EXPIRE_DATE"] < "2030-01-01"]

In [ ]:
ex_cust = sanitized_customer_base.iloc[0]

print(ex_cust)

In [ ]:
last_date = sanitized_customer_base.DATE_SAVED.max()
sanitized_customer_base["CONTRACT_DURATION"] = sanitized_customer_base.apply(lambda x: x.EXPIRE_DATE - x.MEMBER_SINCE if not x.ISACTIVE else last_date - x.MEMBER_SINCE, axis=1)

In [ ]:
sanitized_customer_base.EXPIRE_DATE.describe()

For the subsequent Time Series Analysis it is useful to know the time window that a user has been active. We can simply calculate this by the formula ACTIVE_DURATION = LAST_VISIT - MEMBER_SINCE if LAST_VISIT is not NaN, and ACTIVE_DURATION == 0 days if LAST_VISIT is NaN.

In [ ]:
sanitized_customer_base["ACTIVE_DURATION"] = np.where(sanitized_customer_base.LAST_VISIT.notnull(), sanitized_customer_base.LAST_VISIT - sanitized_customer_base.MEMBER_SINCE, pd.Timedelta("0 days"))
sanitized_customer_base["ACTIVE_DURATION"] = pd.to_timedelta(sanitized_customer_base["ACTIVE_DURATION"])

However, some customers have no last visit entry despite having logins present in the data (see below). For this, we first check how many customers there are without a valid last visit.

In [ ]:
print("There are {0} customers of {1} total customers without a last visit entry.".format(len(sanitized_customer_base[sanitized_customer_base["LAST_VISIT"].isna()].index),len(sanitized_customer_base.index)))

In [ ]:
no_last_visit = sanitized_customer_base[sanitized_customer_base["LAST_VISIT"].isnull()]
users_no_last_visit = no_last_visit.index
no_last_visit

In [ ]:
small_duration = sanitized_customer_base[sanitized_customer_base.CONTRACT_DURATION < pd.Timedelta(days=30)]
sanitized_customer_base.drop(small_duration.index, inplace=True)
sanitized_customer_base

In [ ]:
#passing a list of datatypes for the columns of customer base
reservation_dtypes = {'ID': int, 'START_TIME': str, 'END_TIME': str, 'OBJ_DET_ID': int, 'DATE_SAVED': str, 'DATE_LAST_MODIFIED': str, 
            'INSTRUCTOR_CODE': str, 'SPACE_CODE': str, 'MAX_PEOPLE': int, 'OBJ_CODE': str, 'DESCRIPTION': str, 'LOCATION': int, 
                   'TYPE OF THE CLASS': int, 'Internal/extern': int}

#list of columns that will be parsed as dates
reservation_date_cols = ['START_TIME', 'END_TIME', 'DATE_SAVED', 'DATE_LAST_MODIFIED']

reservation_table = pd.read_excel('../DATA_GYM/RESERVATION.xlsx', parse_dates=reservation_date_cols, index_col=0)

This spreadsheet gives us the overview over the different classes that customers can make a reservation for. They are individual instances of gym events (things like spin classes, Spartan circle etc.). Important columns are:

ID (integer): A simple integer identifier for the timeslot of the event.

OBJ_DET_ID (integer): A complex integer identifier for the reservation encoding the timeslot and type of event at a location (e.g. yoga, spinning). Independent of INSTRUCTOR_CODE and OBJ_CODE, definitely dependent on START_TIME, END_TIME, possibly dependent on SPACE_CODE, MAX_PEOPLE. 

INSTRUCTOR_CODE (string): A string identifying the instructor.

SPACE_CODE (string): A string identifying the specific location space of the event, in terms of LOCATION (see below).

MAX_PEOPLE (integer): The maximum number of participants in the event.

OBJ_CODE (string): A string identifying the login destination. This can be the gym (for people training outside of classes), a course, class ID etc.

START_TIME, END_TIME (datetime objects, format YYYY-MM-DD HH:MM:SS): Start and end dates of the events.

LOCATION (integer): Integer describing the location of the event. Is a weaker identifier than SPACE_CODE.

In [ ]:
reservation_table

In [ ]:
### Import the RESERVATION_DET_xxxxxxx-xxxxxxx spreadsheets and write them into a big-ass dataframe 
file_prefix = "../DATA_GYM/RESERVATION_DET_"
file_type = ".xlsx"
record_numbers = [str(i * int(1e6)) for i in range(1,11)]

record_list = []

#passing a list of datatypes for the columns
login_dtypes = {"ID": int, "RES_ID": int, "DATE_SAVED": str, "DATE_LAST_MODIFIED": str, "CUST_CODE": str, 
            "IS_ARRIVED": str, "LOCATION": int, "COMPUTED_RSV_ID": str}

#columns that are given as dates can be parsed while reading 
login_date_columns = ['DATE_SAVED', 'DATE_LAST_MODIFIED']

#additional values that are recognized as NaN (use-case: ~del_cust as CUST_CODE)
additional_nans = ['~del_cust']

sheet_name = "../DATA_GYM/RESERVATION_DET_1000000.xlsx"

single_sheet_start = time.time()
df = pd.read_excel(sheet_name, index_col=0, dtype=login_dtypes, parse_dates=login_date_columns)
single_sheet_end = time.time()
record_list.append(df)
print("The time for loading the first excel sheet only is {}".format(single_sheet_end - single_sheet_start))

nine_sheets_start = time.time()
for i in range(len(record_numbers) - 1):
    sheet_name = file_prefix + record_numbers[i] + "-" + record_numbers[i+1] + file_type
    df = pd.read_excel(sheet_name, index_col=0, dtype=login_dtypes, parse_dates=login_date_columns)
    record_list.append(df)
nine_sheets_end = time.time()
    
login_data = pd.concat(record_list)
print("The time for loading the other 9 sheets is {}".format(nine_sheets_end - nine_sheets_start))

The RESERVATION_DET_xxxxxxx-xxxxxxx spreadsheets give us the actual time series data. It contains the individual logins of the customers as well as the type of event they attended at which location. Important columns are:

ID (integer): Login ID. Serves as index column in our login_data DataFrame.

RES_ID (integer): Reservation object ID, corresponds to OBJ_DET_ID in the RESERVATION spreadsheet (see above).

DATE_SAVED (datetime object, format DD.MM.YYYY HH:MM): Timestamp of the login that was saved.

CUST_CODE (string, format int1DOTint2): Customer / member ID. Corresponds to the CUST_CODE column of the customer base spreadsheet RES_CARD. 

LOCATION (integer): Location identifier, should correspond to the column of the same name in RESERVATIONS.

In [ ]:
login_data

In [ ]:
pipapo = login_data[login_data["CUST_CODE"].isin(users_no_last_visit)]

pipapo["CUST_CODE"].value_counts()

### Now the workflow is clear: Select a member ID, find all logins associated with the member, put them into a DataFrame and give it a go.

In [ ]:
#example time series using the below customer code
ex_cust_code = "8000938.80"

ex_data = login_data[login_data["CUST_CODE"] == ex_cust_code]

ex_data

In [ ]:
#logins that have a deleted customer code
no_cust_code = login_data[login_data["CUST_CODE"] == "~del_cust"]

#try:
#    login_data_sanitized = pd.read_csv("login_data_sanitized.csv")
#except Exception as e:
login_data_sanitized = login_data.drop(no_cust_code.index)

In [ ]:
#remove logins with NaN customer codes
nan_cust_code = login_data_sanitized[login_data_sanitized["CUST_CODE"].isnull()]

login_data_sanitized.drop(nan_cust_code.index, inplace=True)

In [ ]:
#boolean mask telling which customers from the customer base have actually had a login
user_has_no_login = ~sanitized_customer_base.index.isin(login_data_sanitized["CUST_CODE"])

users_without_login = sanitized_customer_base[user_has_no_login]
print("{0} customers out of {1} total customers have not registered a login.".format(len(users_without_login), len(sanitized_customer_base.index)))

We see that about 15000 of our total (processed) customer base of 158231 do not have a single login. As the analysis of their login data is nonsensical, we drop these users as well.

In [ ]:
sanitized_customer_base.drop(users_without_login.index, inplace=True)

In [ ]:
sanitized_customer_base

In [ ]:
#Convert login dates into timestamps for easier time series analysis
login_date_cols = ["DATE_SAVED", "DATE_LAST_MODIFIED"]

for col in login_date_cols:
    #makes the parser convert wrong timestamps into NaT (not a timestamp)
    login_data_sanitized[col] = pd.to_datetime(login_data_sanitized[col], errors='coerce')

In [ ]:
login_data_sanitized["DATE"] = [ d.date() for d in login_data_sanitized["DATE_SAVED"]]
login_data_sanitized["MONTH"] = [ d.month for d in login_data_sanitized["DATE_SAVED"]]
login_data_sanitized["WEEK"] = [ d.week for d in login_data_sanitized["DATE_SAVED"]]
login_data_sanitized["DAYOFWEEK"] = [ d.dayofweek for d in login_data_sanitized["DATE_SAVED"]]

In [ ]:
login_data_sanitized

In [ ]:
dead_logins = login_data_sanitized[~login_data_sanitized.CUST_CODE.isin(sanitized_customer_base.index)]
dead_logins

In [ ]:
login_data_sanitized.drop(dead_logins.index, inplace=True)
login_data_sanitized

In [ ]:
sanitized_customer_base["NUM_VISITS"] = login_data_sanitized.CUST_CODE.value_counts()

In [ ]:
from utils.stat_utils import calculate_unittest_pvals
#construct result dataframe
stat_types = ["kpss_c", "kpss_ct", "adf_c", "adf_ct"]

results = pd.DataFrame(np.nan, columns=stat_types, index=sanitized_customer_base.index)

#Possible TODO: Make an option to select active range instead of contract duration
last_log_date = login_data_sanitized["DATE_SAVED"].max()   

kwds = {"customer_base": sanitized_customer_base, "last_log_date": last_log_date, "stat_names": stat_types}

In [ ]:
#from tqdm import tqdm
from tqdm.auto import tqdm 
#from pandarallel import pandarallel
#initialize parallel pandas
#pandarallel.initialize(nb_workers=4, progress_bar=False, verbose=verbose)
tqdm.pandas()
results = login_data_sanitized.groupby("CUST_CODE").progress_apply(calculate_unittest_pvals, **kwds)

In [ ]:
augmented_user_base = pd.concat([sanitized_customer_base, results], axis=1)
augmented_user_base.describe()

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss
from utils.time_utils import construct_binary_visit_series

last_log_date = login_data_sanitized["DATE_SAVED"].max()
sample_cust = sanitized_customer_base.sample(n=1).index[0]

#total number of calculated statistics
total_stats = 4

stats = np.full(total_stats, np.nan)

data = login_data_sanitized[login_data_sanitized["CUST_CODE"] == sample_cust]
 
customer = sanitized_customer_base.loc[sample_cust]

#after this, visit_series contains integers 
binary_visit_ts = construct_binary_visit_series(customer, data, last_log_date).astype(float)

#save p-value of kpss test for later significance level setting
stats[0] = kpss(binary_visit_ts, regression="c", nlags="auto")[1]
stats[1] = kpss(binary_visit_ts, regression="ct", nlags="auto")[1]

#save p-value of ADF test 
stats[2] = adfuller(binary_visit_ts, regression="c")[1]
stats[3] = adfuller(binary_visit_ts, regression="ct")[1]

print(stats)

In [ ]:
#login_data_sanitized.groupby(["DATE"]).count()
login_data_sanitized.to_csv("../DATA_GYM/sanitized_login_data.csv")

In [ ]:
# saving the sanitized customer base 
sanitized_customer_base.to_csv("../DATA_GYM/sanitized_customer_base.csv")

In [ ]:
login_counts = login_data_sanitized["CUST_CODE"].value_counts()
login_counts.hist(bins=100)

In [ ]:
login_counts[login_counts <= 50].hist(bins=50)
plt.savefig("../results/visit_modelling/login_hist.jpg", my_dpi=300)

In [ ]:
#Sets the login date as the index of login_data_sanitized. 
#Running twice will result in a KeyError exception
login_data_sanitized = login_data_sanitized.set_index("DATE_SAVED")

In [ ]:
daily_logins.index = pd.to_datetime(daily_logins.index)
weekly_mean_logins = daily_logins.resample('W').sum()

weekly_mean_logins["RES_ID"].plot(linewidth=0.5)
print(weekly_mean_logins["RES_ID"])
print(daily_logins["RES_ID"])

In [ ]:
daily_logins["RES_ID"].diff(periods=1).plot(linewidth=0.5)

In [ ]:
daily_logins["RES_ID"].diff(periods=2).plot(linewidth=0.5)